In [11]:
from openai import OpenAI
import sys
import time
# from prompt import GET_KG_PROMPT
import prompt_template
from utils.myutil import start_ds_client

##将一个txt文档经过prompt调用转为json数据
def get_kj_json_by_txt(txt):
    GET_KG_PROMPT = prompt_template.GET_KG_PROMPT.replace("{txt}",txt)
    client = start_ds_client()
    print("正在发送API请求...")
    try:
        start_time = time.time()
        response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": GET_KG_PROMPT},
        ],
        stream=False
        )
        end_time = time.time()
        print(f"API请求耗时: {end_time - start_time:.2f}秒")
        
        print("正在处理API响应...")
        return response.choices[0].message.content
    except Exception as e:
        print(f"发生错误: {str(e)}", file=sys.stderr)
        sys.exit(1)
    

###
你擅长对中国近代文体的文章进行阅读理解并提取信息。
你的任务是根据给定的文本，从中提取出以下几类信息，这几类信息将被用于制作知识图谱。
###
--1.（'ATTITUDE',主体A，客体B，具体情感，发生时间），此类信息的type被标注为'ATTITUDE',此类信息描述主体A对客体B所持有的情感类型和发生时间，其中主体A必须是人类。如果无法识别出发生时间，则使用空字符串''填充。时间可以精确到小时.
示例：（'ATTITUDE','邹韬奋','感到','张三','愧疚','1925')
--2.('ACT',主体C，动作类型，客体D，动作发生时间，动作的结束时间，动作的持续时间)，此类信息的type被标注为'ACT',此类信息描述主体C对客体D所施加的动作或交互行为。其中主体C和客体D都不必须为人类。如果该动作是不持续性的，则应仅仅具有动作发生时间属性，动作的结束时间和动作的持续时间填充为''。如果动作是持续性的，则应该同时具有动作发生时间、动作的结束时间、和动作的持续时间，如果不能识别出任意一种时间，则填充为''。
示例：('ACT','张三','结婚','李四'，'2002','','')，('ACT','张三','做朋友','李四','2010','2022','12years')
--3.('EVENT'，事件描述，事件牵涉人员列表，事件发生地点，事件发生时间，事件结束时间，事件持续时间)。此类信息的type被标注为'EVENT'。此类信息用于描述事件本体。如果事件是不持续的，则仅有事件发生事件，其余时间字段为''，如果事件为持续性的，则应该具有事件发生时间、事件结束时间，事件持续时间。除了时间描述外的任意信息无法找到，则填充为''。
示例：('EVENT','2022年，张三和李四在酒店举办了婚礼','['张三','李四']','酒店','2022','','')
--4.('REASON',事件A,事件B),此类信息用于描述事件本体之间的因果关系。此类信息的type被标注为'REASON'。示例：('REASON','2022年，张三和李四在酒店举办了婚礼','酒店营业额上升')
--5.('RELATION'，主体A，关系，客体B，关系发生时间，关系结束时间，关系持续时间)。此类信息用于描述主体A和客体B的人际、职务、功能、结构上的关系，此

{{
    'type':'ATTITUE',
    'subject':'邹韬奋',
    'object':'张三',
    'attitude_type':'愧疚',
    'start_time':'上午'
    }},
    {{
    'type':'ACT',
    'subject':'张三'
    'action':'做朋友',
    'object':'李四',
    'start_time':'2010-01-01',
    'end_time':'2022-01-01',
    'endure_time':'12 years 00:00:00'
    }},
    {{
    'type':'EVENT',
    'descrpition':'2022年，张三和李四在酒店举办了婚礼',
    'person_infer':['张三','李四'],
    'position':'酒店',
    'start_time':'2022-01-01 00:00:00',
    'end_time':'',
    'endure_time':''
    }},
    {{
     'type':'REASON',
     'reason':'2022年，张三和李四在酒店举办了婚礼',
     'effect':'酒店营业额上升'
    }},
    {{
        'type':'RELATION',
        'subject':'纽约',
        'relationship':'位于',
        'object':'美国',
        start_time:'',
        end_time:'',
        endure_time:''
    }},
    {{
        'type':'PROPERTY',
        'subject':'张三',
        'property_type':'性别',
        'preperty_value':'男'
    }}

In [18]:
###导入json数据
import json
with open('test_source/deepseek_json_20250612_47efda.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
type(json_data['msg'])

list

In [27]:
###将json数据写入neo4j数据库
from neo4j import GraphDatabase
import json

###创建neo4j连接对象
def get_neo4j_driver():
    uri = "bolt://localhost:7687"  # Neo4j数据库地址
    username = "neo4j"  # 用户名
    password = "wzdshjw123"  # 密码
    try:
        # 连接数据库
        driver = GraphDatabase.driver(uri, auth=(username, password))
        print("成功连接到Neo4j数据库")
        return driver
    except Exception as e:
        print(f"写入Neo4j时发生错误: {str(e)}")
        sys.exit(1)

### 在一个session中执行query
def excute_query(driver,query):
    try:
        with driver.session() as session:
            session.run(query)
        return('neo4jquery成功')
    except Exception as e:
        print(f"Neo4jquery发生错误: {str(e)}")
        sys.exit(1)

In [80]:
data_list[1]

{'type': 'ATTITUDE',
 'subject': '检察官',
 'object': '邹韬奋等',
 'attitude_type': '同情',
 'start_time': '廿七日'}

In [79]:
###将json数据写入neo4j数据库
data_list = json_data['msg']
#首先确保唯一性约束
constraint_query = """CREATE CONSTRAINT person_name_unique IF NOT EXISTS
FOR (p:Person) REQUIRE p.name IS UNIQUE;"""
neo4j_driver = get_neo4j_driver()
excute_query(neo4j_driver,constraint_query)
# neo4j_driver.close()
for data in data_list:
    try:    
        if data['type']=='ATTITUDE':
            subject = data['subject']
            object = data['object']
            attitude_type = data['attitude_type']
            start_time = data['start_time']
            query = f"""
            merge (p1:Person {{name:'{subject}'}})\
            merge(p2:Person {{name:'{subject}'}})\
            WITH p1, p2 WHERE id(p1) <> id(p2)\
            merge (p1)-[:ATTITUDE {{attitude_type:'{attitude_type}',start_time:'{start_time}'}}]->(p2)
            """
        elif data['type']=='ACT':
            subject = data['subject']
            object = data['object']
            action = data['action']
            start_time = data['start_time']
            end_time = data['end_time']
            endure_time = data['endure_time']
            query = f"""
            merge (p1:Person {{name:'{subject}'}})
            merge (p2:Person {{name:'{object}'}})
            WITH p1, p2 WHERE id(p1) <> id(p2)\
            merge (p1)-[:ACT {{action:'{action}',\
                start_time:'{start_time}',\
                end_time:'{end_time}',\
                endure_time:'{endure_time}'\
                }}]->(p2)
            """
        elif data['type']=='EVENT':
            description  = data["description"]
            position     = data["position"]
            start_time   = data["start_time"]
            end_time     = data["end_time"]
            endure_time  = data["endure_time"]

            cypher_lines = [
                f"""MERGE (ev:Event {{
                    name:        '{description}',
                    start_time:  '{start_time}',
                    end_time:    '{end_time}',
                    endure_time: '{endure_time}',
                    position:    '{position}'
                }})"""
            ]

            for idx, person in enumerate(data["person_infer"]):
                # 用唯一变量名避免覆盖：p0 / p1 / …
                cypher_lines.append(
                    f"MERGE (p{idx}:Person {{name:'{person}'}})"
                    f"MERGE (ev)-[:REFER_TO]->(p{idx})"
                )

            query = "\n".join(cypher_lines) + ";"

            # description= data['description']
            # position=data['position']
            # start_time = data['start_time']
            # end_time = data['end_time']
            # endure_time = data['endure_time']
            # query = ""
            # for people in data['person_infer']:
            #     query +=f"""
            #     merge (:Event {{name:'{description}',start_time:'{start_time}',\
            #         end_time:'{end_time}',endure_time:'{endure_time}',position:'{position}'}})\
            #         -[:REFER_TO]->(:Person {{name:'{people}'}})
            # """
        elif data['type']=='REASON':
            reason = data['reason']
            effect = data['effect']
            query = f"""
            merge (e1:Event {{name:'{reason}'}})
            merge (e2:Event {{name:'{effect}'}})
            WITH e1, e2 WHERE id(e1) <> id(e2)\
            merge (e1)-[:CAUSE]->(e2)
            """
        elif data['type']=='RELATION':
            subject = data['subject']
            object = data['object']
            relation = data['relationship']
            start_time = data['start_time']
            end_time = data['end_time']
            endure_time = data['endure_time']
            query = f"""
            merge (p1:Person {{name:'{subject}'}})
            merge (p2:Person {{name:'{object}'}})
            WITH p1, p2 WHERE id(p1) <> id(p2)\
            merge (p1)-[:RELATION_TO {{name:'{relation}',\
                start_time:'{start_time}',\
                end_time:'{end_time}',endure_time:'{endure_time}'}}]->(p2)"""
        elif data['type']=='PROPERTY':
            subject = data['subject']
            property_type= data['property_type']
            property_value = data['property_value']
            query = f"""
            merge (p1:Person {{name:'{subject}'}})
            merge (p2:Person {{name:'{property_value}'}})
            WITH p1, p2 WHERE id(p1) <> id(p2)\
            merge (p1)-[:PROPERTY {{property_type:'{property_type}'}}]->(p2)
            """
        print(query)
        # neo4j_driver = get_neo4j_driver()
        excute_query(neo4j_driver,query)
        # neo4j_driver.close()
    except Exception as e:
        print(f"发生错误: {str(e)}", file=sys.stderr)
        sys.exit(1)
neo4j_driver.close()  

成功连接到Neo4j数据库


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 2, column: 112, offset: 112} for query: "\n            merge (p1:Person {name:'法警'})            merge(p2:Person {name:'法警'})            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:ATTITUDE {attitude_type:'同情',start_time:''}]->(p2)\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 2, column: 122, offset: 122} for query: "\n            merge (p1:Person {name:'法警'})            merge(p2:Person {name:'法警'})         


            merge (p1:Person {name:'法警'})            merge(p2:Person {name:'法警'})            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:ATTITUDE {attitude_type:'同情',start_time:''}]->(p2)
            

            merge (p1:Person {name:'检察官'})            merge(p2:Person {name:'检察官'})            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:ATTITUDE {attitude_type:'同情',start_time:'廿七日'}]->(p2)
            

            merge (p1:Person {name:'邹韬奋'})            merge(p2:Person {name:'邹韬奋'})            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:ATTITUDE {attitude_type:'感激',start_time:'廿七日'}]->(p2)
            

            merge (p1:Person {name:'邹韬奋'})            merge(p2:Person {name:'邹韬奋'})            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:ATTITUDE {attitude_type:'赞扬',start_time:'廿七日'}]->(p2)
            

            merge (p1:Person {name:'侦探'})            merge(p2:Person {name:'侦探'})            WITH p1, p2 WHERE id(p1

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 31, offset: 134} for query: "\n            merge (e1:Event {name:'访问者非常多导致客厅常告客满'})\n            merge (e2:Event {name:'当局禁止接见非家属亲友'})\n            WITH e1, e2 WHERE id(e1) <> id(e2)            merge (e1)-[:CAUSE]->(e2)\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 41, offset: 144} for query: "\n            merge (e1:Event {name:'访问者非常多导致客厅常告客满'})\n            merge (e2:Event {name:'当局禁止接见非家属亲友'})\n 


            merge (p1:Person {name:'侦探'})
            merge (p2:Person {name:'邹韬奋等'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:RELATION_TO {name:'监视',                start_time:'进入公安局后',                end_time:'',endure_time:'持续数日'}]->(p2)

            merge (p1:Person {name:'武装同志'})
            merge (p2:Person {name:'邹韬奋等'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:RELATION_TO {name:'监视',                start_time:'进入公安局后',                end_time:'',endure_time:'持续数日'}]->(p2)

            merge (p1:Person {name:'邹韬奋'})
            merge (p2:Person {name:'章先生'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:RELATION_TO {name:'同被关押',                start_time:'廿七日',                end_time:'',endure_time:'持续数日'}]->(p2)

            merge (p1:Person {name:'邹韬奋'})
            merge (p2:Person {name:'沈先生等四人'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:RELATION_TO {name:'同被关

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 31, offset: 117} for query: "\n            merge (p1:Person {name:'武装同志'})\n            merge (p2:Person {name:'中国'})\n            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:PROPERTY {property_type:'国籍'}]->(p2)\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 41, offset: 127} for query: "\n            merge (p1:Person {name:'武装同志'})\n            merge (p2:Person {name:'中国'})\n           


            merge (p1:Person {name:'武装同志'})
            merge (p2:Person {name:'中国'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:PROPERTY {property_type:'国籍'}]->(p2)
            

            merge (p1:Person {name:'沈先生等人的房间'})
            merge (p2:Person {name:'二层楼'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:PROPERTY {property_type:'位置'}]->(p2)
            

            merge (p1:Person {name:'沈先生等人的房间'})
            merge (p2:Person {name:'四个小铁床、露台、屏风、圆桌、椅子'})
            WITH p1, p2 WHERE id(p1) <> id(p2)            merge (p1)-[:PROPERTY {property_type:'设施'}]->(p2)
            


In [ ]:
neo4j_driver = get_neo4j_driver()
#首先确保唯一性约束
constraint_query = """CREATE CONSTRAINT person_name_unique IF NOT EXISTS
FOR (p:Person) REQUIRE p.name IS UNIQUE;"""
excute_query(neo4j_driver,constraint_query)
for data in data_list:
    try:    
        if data['type']=='ATTITUDE':
            subject = data['subject']
            object = data['object']
            attitude_type = data['attitude_type']
            start_time = data['start_time']
            query = f"""
            merge (:Person {{name:'{subject}'}})
            -[:ATTITUDE {{attitude_type:'{attitude_type}',start_time:'{start_time}'}}]->(:Person {{name:'{subject}'}})
            """
        elif data['type']=='ACT':
            subject = data['subject']
            object = data['object']
            action = data['action']
            start_time = data['start_time']
            end_time = data['end_time']
            endure_time = data['endure_time']
            query = f"""
            merge (:Person {{name:'{subject}'}})-[:ACT {{action:'{action}',\
                start_time:'{start_time}',\
                end_time:'{end_time}',\
                endure_time:'{endure_time}'\
                }}]->(:Person {{name:'{object}'}})
            """
        elif data['type']=='EVENT':
            description= data['description']
            position=data['position']
            start_time = data['start_time']
            end_time = data['end_time']
            endure_time = data['endure_time']
            query = ""
            for people in data['person_infer']:
                query +=f"""
                merge (:Event {{name:'{description}',start_time:'{start_time}',\
                    end_time:'{end_time}',endure_time:'{endure_time}',position:'{position}'}})\
                    -[:REFER_TO]->(:Person {{name:'{people}'}})
            """
        elif data['type']=='REASON':
            reason = data['reason']
            effect = data['effect']
            query = f"""
            merge (:Event {{name:'{reason}'}})-[:CAUSE]->(:Event {{name:'{effect}'}})
            """
        elif data['type']=='RELATION':
            subject = data['subject']
            object = data['object']
            relation = data['relationship']
            start_time = data['start_time']
            end_time = data['end_time']
            endure_time = data['endure_time']
            query = f"""
            merge (:Person {{name:'{subject}'}})-[:RELATION_TO {{name:'{relation}',\
                start_time:'{start_time}',\
                end_time:'{end_time}',endure_time:'{endure_time}'}}]->(:Person {{name:'{object}'}})
            """
        elif data['type']=='PROPERTY':
            subject = data['subject']
            property_type= data['property_type']
            property_value = data['property_value']
            query = f"""
            merge (:Person {{name:'{subject}'}})-[:PROPERTY {{property_type:'{property_type}'}}]->(:Person {{name:'{property_value}'}})
            """
        excute_query(neo4j_driver,query)
    except Exception as e:
        print(f"发生错误: {str(e)}", file=sys.stderr)
        sys.exit(1)
    

成功连接到Neo4j数据库
Neo4jquery发生错误: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(72) already exists with label `Person` and property `name` = '法警'}


SystemExit: 1

d:\python_envs\ai_project\Lib\site-packages\IPython\core\interactiveshell.py:3680: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
